# Evaluating Draft and Lineup Performance
"You can't win your league with your draft but you can certainly lose it." An r/fantasyfootball trope as old as time itself. Seems that winning a fantasy championship relies on a few factors including a solid draft, smart trades, efficient waiver pickups, and of course, some dumb luck. Here's my attempt at evaluating and visualizing any of it.

In [1]:
{"tags": ["hide-cell"]}
import warnings; warnings.simplefilter('ignore')

# Modules
import numpy as np
import pandas as pd
from espn_api.football import League
from espn_api.football import constant

import altair as alt

# League Parameters
league_id = 298982
swid = '{D825016D-4C3D-4575-B33C-2C2277B026F0}'
espn_s2 = 'AEAmlIpZf7Zd7LTyvSFyl9k3zui2t4hQwlVtJM8WK3Lmx33eWNAUq32gR9NK98ZjqXWIEsK3NETxdtcctstCwGu45Mx9QOM9wIeB9KBTALOs8wg512Me2GSTnw3MCQL8bAeTp16w0xkggMxdmDGX9BX6nS2dKDx5OfjEIFLsgnntd3CW%2BmYsMnCywwMpQQZQkigiNNM54cM7OmivIq2kGkZY%2BJEhquYe%2FHnSRhBa9f052nnEmYn0fAzax8RHr3boSL9UASeMp2B8dLubfQd83Bj%2B%2BwBJPpeGyxi36lwJPtMXjg%3D%3D'
league_start = 2011

In [2]:
{"tags": ["hide-cell"]}

all_comps = []

# pull league info
league = League(league_id, 2024, espn_s2, swid)

# how many weeks of data?
num_weeks = league.current_week

def find_original_lineup(lineup):
    og_lineup_pos = [player.lineupSlot for player in lineup]
    og_lineup_name = [player.name for player in lineup]
    og_lineup_score = [player.points for player in lineup]

    og_lineup_df = pd.DataFrame({
        'og_lineup_pos': og_lineup_pos,
        'og_lineup_name': og_lineup_name,
        'og_lineup_score': og_lineup_score,
    })

    # and thier score
    og_score = np.sum([player.points for player in lineup if player.lineupSlot not in ['BE', 'IR']])

    return og_lineup_df, og_score

# loop through weeks for each weeks' box score
for i in range(num_weeks):
    week_box_scores = league.box_scores(week = i + 1)

    # count the number of matchups in the week
    num_of_matchups = len(week_box_scores)

    # loop through matchups and retrieve box scores
    for j in range(num_of_matchups):
        home_team = week_box_scores[j].home_team
        home_lineup = week_box_scores[j].home_lineup

        manager = home_team.owners[0]['firstName'] + ' ' + home_team.owners[0]['lastName']
        
        og_lineup_df, og_score = find_original_lineup(home_lineup)

        manager = home_team.owners[0]['firstName'] + ' ' + home_team.owners[0]['lastName']

        # add other identifiable parameters
        og_lineup_df['year'] = 2024
        og_lineup_df['manager'] = manager
        og_lineup_df['manager'] = og_lineup_df['manager'].str.title()
        og_lineup_df['week'] = i + 1

        # append the dataframes
        all_comps.append(og_lineup_df)

    for k in range(num_of_matchups):
        away_team = week_box_scores[k].away_team
        away_lineup = week_box_scores[k].away_lineup

        manager = away_team.owners[0]['firstName'] + ' ' + away_team.owners[0]['lastName']
        
        og_lineup_df, og_score = find_original_lineup(away_lineup)

        manager = away_team.owners[0]['firstName'] + ' ' + away_team.owners[0]['lastName']

        # add other identifiable parameters
        og_lineup_df['year'] = 2024
        og_lineup_df['manager'] = manager
        og_lineup_df['manager'] = og_lineup_df['manager'].str.title()
        og_lineup_df['week'] = i + 1

        # append the dataframes
        all_comps.append(og_lineup_df)

all_lineup_df = pd.concat(all_comps).reset_index(drop = True)

In [3]:
{"tags": ["hide-cell"]}

# Helper Functions
def find_original_lineup(lineup):
    og_lineup_pos = [player.lineupSlot for player in lineup]
    og_lineup_name = [player.name for player in lineup]

    og_lineup_df = pd.DataFrame({
        'og_lineup_pos': og_lineup_pos,
        'og_lineup_name': og_lineup_name
    })

    # remove non-starters
    og_lineup_df = og_lineup_df[~og_lineup_df['og_lineup_pos'].isin(['BE', 'IR'])].reset_index(drop = True)

    # and thier score
    og_score = np.sum([player.points for player in lineup if player.lineupSlot not in ['BE', 'IR']])
    projected_score = np.sum([player.projected_points for player in lineup if player.lineupSlot not in ['BE', 'IR']])

    return og_lineup_df, og_score, projected_score

def find_optimal_lineup(week, positions, lineup):
    # empty lists to append to
    optimal_positions = []
    optimal_players = []

    # 1. outer loop: go through all the positions
    for position in positions:

        # skip the non-starters
        if position in ['BE', 'IR']:
            continue
        if positions[position] != 0:
            n_players = positions[position]
            
            # save the positions and counts
            optimal_positions.append(np.repeat(position, n_players))

            # 2. inner loop: find highest player for each position
            eligible_players = []
            for player in lineup:
                if position in player.eligibleSlots:

                    # save the player name
                    eligible_players.append(player)

            # sort players by points scored; keep roster available number
            eligible_players = sorted(eligible_players, key = lambda x: x.points, reverse = True)[:n_players]
            optimal_players.extend(eligible_players)

            # remove players already used
            for player in eligible_players:
                lineup.remove(player)

    optimal_lineup_df = pd.DataFrame({
        'optimal_positions': sum([pos.tolist() for pos in optimal_positions], []),
        'optimal_players': [player.name for player in optimal_players]
    })
    optimal_lineup_df['week'] = week

    # output the score as well
    optimal_score = np.sum(player.points for player in optimal_players)

    return optimal_lineup_df, optimal_score

def combine_og_optimal(og_lineup_df, optimal_lineup_df):
    og_lineup_df['og_lineup_pos'] = og_lineup_df['og_lineup_pos'].astype('category')
    og_lineup_df['og_lineup_pos'] = og_lineup_df['og_lineup_pos'].cat.set_categories(optimal_lineup_df['optimal_positions'].unique())
    og_lineup_df = og_lineup_df.sort_values('og_lineup_pos').reset_index(drop = True)

    comp_df = pd.concat([optimal_lineup_df, og_lineup_df], axis = 1)

    return comp_df

def gather_all_optimal(league, all_comps, all_scores):
    num_weeks = league.current_week
    # no need to loop for positions
    positions = league.settings.position_slot_counts

    # loop through all the weeks we have so far
    for j in range(num_weeks - 1):

        # Access a week of box scores
        week_box_scores = league.box_scores(week = j + 1)
        
        # loop through all the home & away teams
        num_of_matchups = len(week_box_scores)

        for i in range(num_of_matchups):
            home_team = week_box_scores[i].home_team
            home_lineup = week_box_scores[i].home_lineup

            manager = home_team.owners[0]['firstName'] + ' ' + home_team.owners[0]['lastName']
            
            # make a copy to not reset values
            home_lineup2 = home_lineup.copy()

            # find optimal & original lineups & scores for the team
            optimal_lineup_df, optimal_score = find_optimal_lineup(week = j + 1,
                                                                   positions = positions,
                                                                   lineup = home_lineup2)
            og_lineup_df, og_score, projected_score = find_original_lineup(home_lineup)

            # another data frame to host scores
            oo_score_df = pd.DataFrame({
                'manager': np.repeat(manager, 3),
                'week': np.repeat(j + 1, 3),
                'score': [og_score, optimal_score - og_score, projected_score],
                'type': ['original', 'optimal', 'projected']
            })

            optimal_lineup_df['manager'] = manager

            # join the dataframes
            comp_df = combine_og_optimal(og_lineup_df, optimal_lineup_df)

            # append the dataframes
            all_comps.append(comp_df)
            all_scores.append(oo_score_df)

        for i in range(num_of_matchups):
            away_team = week_box_scores[i].away_team
            away_lineup = week_box_scores[i].away_lineup

            manager = away_team.owners[0]['firstName'] + ' ' + away_team.owners[0]['lastName']
            
            # make a copy to not reset values
            away_lineup2 = away_lineup.copy()

            # find optimal & original lineups & scores for the team
            optimal_lineup_df, optimal_score = find_optimal_lineup(week = j + 1,
                                                                   positions = positions,
                                                                   lineup = away_lineup2)
            og_lineup_df, og_score, projected_score = find_original_lineup(away_lineup)

            # another data frame to host scores
            oo_score_df = pd.DataFrame({
                'manager': np.repeat(manager, 3),
                'week': np.repeat(j + 1, 3),
                'score': [og_score, optimal_score - og_score, projected_score],
                'type': ['original', 'optimal', 'projected']
            })

            optimal_lineup_df['manager'] = manager

            # join the dataframes
            comp_df = combine_og_optimal(og_lineup_df, optimal_lineup_df)

            # append the dataframes
            all_comps.append(comp_df)
            all_scores.append(oo_score_df)

    all_comps_df = pd.concat(all_comps).reset_index(drop = True)
    all_scores_df = pd.concat(all_scores).reset_index(drop = True)

    return all_comps_df, all_scores_df

def create_master_data():
    # get all the possible years' worth of data
    # current_year = datetime.date.today().year
    current_year = 2024
    all_years = np.arange(league_start, current_year + 1)

    # remove 2018 as its a problem (TO FIX)
    index_2018 = np.where(all_years == 2018)
    if index_2018[0].shape[0] == 1:
        all_years = np.delete(all_years, index_2018)

    # get data for all the years in raw format
    leagues = []
    for year in all_years:
        league = League(league_id, year, espn_s2, swid)
        leagues.append(league)

    # loop through each league and grab game-level data
    scores_df = []

    # use the loop to grab the final standings too
    standings_df = []

    # use the loop to grab the acquisition counts too
    acq_df = []

    # use the loop to grab the draft results
    draft_board = []

    # use the loop to grab player data
    player_lookup = []

    # use the loop to grab optimal comparison data
    all_comps = []
    all_scores = []

    # outer loop to loop through all the leagues/years
    for league in leagues:
        season_weeks = league.settings.reg_season_count
        teams = league.teams
        team_ids = [team.team_id for team in teams]

        # metrics for the draft board
        num_picks = len(league.draft)

        # inner loop to loop through the teams in each year
        for id in team_ids:
            team_data = league.get_team_data(id)

            # some managers' data has been purged
            if team_data is None:
                continue
            else:
                opponent_list = pd.DataFrame({'opponent': [opp.owners[0]['firstName'] + ' ' + opp.owners[0]['lastName'] for opp in team_data.schedule],
                                              'outcome': team_data.outcomes,
                                              'points_for': team_data.scores,
                                              'mov': team_data.mov})
                opponent_list['manager'] = team_data.owners[0]['firstName'] + ' ' + team_data.owners[0]['lastName']
                opponent_list['game_type'] = 'postseason'
                opponent_list['week'] = np.arange(opponent_list.shape[0]) + 1
                scores_df.append(opponent_list)
                opponent_list.iloc[:season_weeks, 5] = 'season'
                opponent_list['year'] = league.year

                # acquisition data
                team_id = team_data.owners[0]['firstName'] + ' ' + team_data.owners[0]['lastName']
                pickups = team_data.acquisitions
                trades = team_data.trades
                faab_used = team_data.acquisition_budget_spent
                acq_list = pd.DataFrame({'team_id': [team_id],
                                         'pickups': [pickups],
                                         'trades': [trades],
                                         'faab_used': [faab_used],
                                         'year': [league.year]})
                acq_df.append(acq_list)

        # accumulate the standings from each year
        standings = [team.owners[0]['firstName'] + ' ' + team.owners[0]['lastName'] for team in league.standings()]
        standings_num = np.arange(len(standings)) + 1
        df_standings = pd.DataFrame({'Manager': standings, 'Result': standings_num})
        df_standings['Year'] = league.year
        standings_df.append(df_standings)

        # collect draft values here
        for i in range(num_picks):
            draft_pick = league.draft[i]
            round = draft_pick.round_num
            pick = draft_pick.round_pick
            player = draft_pick.playerName
            manager = manager = draft_pick.team.owners[0]['firstName'] + ' ' + draft_pick.team.owners[0]['lastName']

            # combine all into an appendable data frame
            pick_df = pd.DataFrame({'round': [round],
                                    'pick': [pick],
                                    'player': [player],
                                    'manager': [manager],
                                    'year': [league.year]})

            draft_board.append(pick_df)

        # collect player data here
        dat = league.espn_request.get_pro_players()
        for player_info in dat:
            if 'eligibleSlots' not in player_info.keys():
                continue
            # position_id = player_info['eligibleSlots'][0]

            # filter for only first eligible player spot that isnt a combo
            for pos in player_info['eligibleSlots']:
                if (pos != 25 and '/' not in constant.POSITION_MAP[pos]) or '/' in player_info['fullName']:
                    position = constant.POSITION_MAP[pos]
                    break

            # remainder of player data
            player = player_info['fullName']
            pi = pd.DataFrame({'position': [position],
                               'player': [player],
                               'year': [league.year]})
            player_lookup.append(pi)

        if league.year == current_year:
            all_comps_df, all_scores_df = gather_all_optimal(league, all_comps, all_scores)

    # all the concatenations
    scores_df = pd.concat(scores_df)
    scores_df['manager'] = scores_df['manager'].str.title()
    scores_df['opponent'] = scores_df['opponent'].str.title()
    scores_df['points_against'] = scores_df['points_for'] - scores_df['mov']

    standings_df = pd.concat(standings_df)
    standings_df['Manager'] = standings_df['Manager'].str.title()

    acq_df = pd.concat(acq_df)
    acq_df['team_id'] = acq_df['team_id'].str.title()

    draft_board_df = pd.concat(draft_board)
    draft_board_df['manager'] = draft_board_df['manager'].str.title()

    all_scores_df['manager'] = all_scores_df['manager'].str.title()
    all_comps_df['manager'] = all_comps_df['manager'].str.title()

    # concatenate and remove duplicates (defense, etc.)
    player_lookup_df = pd.concat(player_lookup)
    player_lookup_df['row_num'] = player_lookup_df.groupby(['player', 'year'], as_index = False).cumcount() + 1
    player_lookup_df = player_lookup_df[player_lookup_df['row_num'] == 1]

    # add draft position to the draft
    draft_board_df['player_pos'] = draft_board_df['player'] + ' (' + draft_board_df['round'].astype(str) + '.' + draft_board_df['pick'].astype(str) + ')'

    # add playing position to the draft
    draft_board_df = pd.merge(draft_board_df,
                              player_lookup_df,
                              how = 'left',
                              on = ['player', 'year'])

    return scores_df, standings_df, acq_df, draft_board_df, all_comps_df, all_scores_df

scores_df, standings_df, acq_df, draft_board_df, all_comps_df, all_scores_df = create_master_data()

In [4]:
{"tags": ["hide-cell"]}

lineups_2024 = all_lineup_df[all_lineup_df['og_lineup_pos'].isin(['QB', 'RB', 'WR', 'RB/WR', 'TE', 'RB/WR/TE', 'K', 'D/ST'])]
lineups_2024 = pd.merge(
    lineups_2024,
    scores_df.loc[:, ['manager', 'week', 'year', 'points_for', 'outcome']], 
    how = 'left', 
    on = ['manager', 'week', 'year']
)
lineups_2024 = lineups_2024[lineups_2024['outcome'] == 'W'].dropna()
lineups_2024['win_shares'] = lineups_2024['og_lineup_score'] / lineups_2024['points_for']

## Win Shares
The first metric I'm looking at is Win Shares. An **advanced statistic** in NBA and MLB circles, this metric is usually a box-score conglomerate. For the sake of fantasy football, our box scores are relatively straightforward - the only value of interest is points. As a result, positive Win Shares only count each player's point contribution to each win and is then summed up. It's not a groundbreaking formula by any means but it sheds light on which players were the most dominant in combination with other players on a team. Win Shares show context of the team and manager rather than focusing just on total points.

In [5]:
{"tags": ["hide-input"]}

overall_win_shares = (
    lineups_2024.loc[:, ['og_lineup_name', 'win_shares']]
    .groupby('og_lineup_name', as_index = False)
    .sum()
    .sort_values('win_shares', ascending = False)
)

overall_win_shares_plot = (
    alt.Chart(
        overall_win_shares,
        title = 'Distribution of Player Win Shares'
    ).mark_bar().encode(
        x = alt.X('win_shares:Q', bin = True).title('Win Shares'),
        y = alt.Y('count()').title('Number of Players')
    )
)
overall_win_shares_plot

alt.Chart(...)

In [6]:
{"tags": ["hide-input"]}
overall_win_shares.rename(columns = {'og_lineup_name': 'Player Name', 'win_shares': 'Win Shares'}).head(10)

,Player Name,Win Shares
130,Josh Allen,2.260749
150,Lamar Jackson,2.152806
75,Derrick Henry,1.794700
186,Saquon Barkley,1.575060
122,Joe Burrow,1.538971
107,Jalen Hurts,1.518350
99,Ja'Marr Chase,1.484233
101,Jahmyr Gibbs,1.464843
132,Josh Jacobs,1.446361
125,Jonathan Taylor,1.416881


These are the top 10 players in terms of Win Shares. Making sense of the table is straightforward. Saquon contributed, by himself, about 1.58 wins to Alec's team. This does take into consideration trades and such - Jordan lost week 1 with Saquon, thus Saquon contributed no win shares to Jordan's team.

And every manager's highest Win Share player:

In [10]:
{"tags": ["hide-input"]}

overall_win_shares_by_manager = (
    lineups_2024.loc[:, ['og_lineup_name', 'manager', 'win_shares']]
    .groupby(['og_lineup_name', 'manager'], as_index = False)
    .sum()
    .sort_values(['win_shares', 'manager'], ascending = False)
    .groupby('manager', as_index = False)
    .head(1)
    .rename(columns = {'og_lineup_name': 'Player Name', 'win_shares': 'Win Shares', 'manager': 'Manager'})
    .iloc[:, [1, 0, 2]]
)
overall_win_shares_by_manager

,Manager,Player Name,Win Shares
168,Kooper Knutson,Josh Allen,2.260749
194,Tyler Mcwilliams,Lamar Jackson,2.152806
97,Brian Babcock,Derrick Henry,1.794700
238,Alec Sa,Saquon Barkley,1.575060
160,Jordan Fladger,Joe Burrow,1.538971
141,Michael Mount,Jalen Hurts,1.518350
130,Michael Oatman,Ja'Marr Chase,1.484233
133,Brian Badillo,Jahmyr Gibbs,1.464843
163,Nikhil Tellakula,Jonathan Taylor,1.416881
154,Sirpi Nackeeran,Jayden Daniels,0.994838


We can use this same logic and apply it to everyone's draft. Not starting lineups week-over-week (that would just sum up to total wins), but who drafted the best team? No waiver wire pickups, including injury luck, etc. - who drafted the best?

Spoilers, it was Oatman.

In [11]:
draft_win_shares = (draft_board_df[draft_board_df['year'] == 2024]
                    .drop(['round', 'pick', 'year', 'player_pos', 'row_num'], axis = 1)
                    .merge(lineups_2024.loc[:, ['og_lineup_pos', 'og_lineup_name', 'win_shares']],
                           how = 'left',
                           left_on = ['player', 'position'],
                           right_on = ['og_lineup_name', 'og_lineup_pos'])
                    .loc[:, ['manager', 'win_shares']]
                    .groupby('manager', as_index = False)
                    .sum()
                    .sort_values('win_shares', ascending = False)
                    .rename(columns = {'manager': 'Manager', 'win_shares': 'Win Shares'})
)
draft_win_shares

,Manager,Win Shares
8,Michael Oatman,8.480125
11,Tyler Mcwilliams,7.659611
1,Brian Babcock,7.153210
6,Kooper Knutson,7.080386
0,Alec Sa,6.622765
9,Nikhil Tellakula,6.604814
7,Michael Mount,6.272272
5,Jordan Fladger,6.100912
2,Brian Badillo,4.742618
4,Chris Lamelas,4.103016


## VORP
Another fun advanced metric to evaluate players is VORP (Value Over Replacement Player), or WAR (Wins Above Replacement). 

In [ ]:
def create_player_base(week):

    # all free agents
    all_player_list = league.free_agents()
    week_box_scores = league.box_scores(week = week)

    # loop through matchups and retrieve box scores
    for j in range(num_of_matchups):
        home_lineup = week_box_scores[j].home_lineup
        away_lineup = week_box_scores[j].away_lineup

        # append the lineup
        all_player_list = all_player_list + home_lineup
        all_player_list = all_player_list + away_lineup

    return all_player_list

all_players = create_player_base(17)

# Dataframe of everyone that has been on a roster at some point
total_points = [player.total_points for player in all_players]
name = [player.name for player in all_players]
position = [player.position for player in all_players]
player_season_points = pd.DataFrame({'total_points': total_points, 'name': name, 'position': position})

player_points = pd.merge(player_season_points, 
                         all_lineup_df, 
                         how = 'inner', 
                         left_on = 'name', 
                         right_on = 'og_lineup_name')\
                .loc[:, ['name', 'position', 'og_lineup_score']]

# Sorting Appropriately
player_ranks = (
    player_points.groupby(['name', 'position'], as_index = False)
    .sum()
    .sort_values(['position', 'og_lineup_score'], ascending = False)
    .reset_index(drop = True)
)

In [58]:
wrs = player_ranks[player_ranks['position'] == 'WR']
wrs = wrs.reset_index().rename(columns = {'index': 'rank'})
wrs['rank'] = wrs['rank'] + 1

,rank,name,position,og_lineup_score
0,1,Ja'Marr Chase,WR,252.0
1,2,Justin Jefferson,WR,201.0
2,3,Amon-Ra St. Brown,WR,188.0
3,4,Brian Thomas Jr.,WR,178.0
4,5,Terry McLaurin,WR,168.0
...,...,...,...,...
66,67,Tyler Lockett,WR,21.0
67,68,Wan'Dale Robinson,WR,15.0
68,69,Jalen Coker,WR,6.0
69,70,DeMario Douglas,WR,2.0


In [65]:
wr1 = wrs.loc[:11, :]
wr1['avg'] = np.mean(wr1['og_lineup_score'])
wr1['vorp'] = wr1['og_lineup_score'] - wr1['avg']
wr1

,rank,name,position,og_lineup_score,avg,vorp
0,1,Ja'Marr Chase,WR,252.0,167.0,85.0
1,2,Justin Jefferson,WR,201.0,167.0,34.0
2,3,Amon-Ra St. Brown,WR,188.0,167.0,21.0
3,4,Brian Thomas Jr.,WR,178.0,167.0,11.0
4,5,Terry McLaurin,WR,168.0,167.0,1.0
5,6,CeeDee Lamb,WR,152.0,167.0,-15.0
6,7,Mike Evans,WR,150.0,167.0,-17.0
7,8,Malik Nabers,WR,145.0,167.0,-22.0
8,9,Drake London,WR,143.0,167.0,-24.0
9,10,Jameson Williams,WR,143.0,167.0,-24.0


In [70]:
draft_board_df[(draft_board_df['year'] == 2024) & (draft_board_df['manager'] == 'Alec Sa') & (draft_board_df['round'] <= 9)]

,round,pick,player,manager,year,player_pos,position,row_num
1870,1,1,Kyler Murray,Alec Sa,2024,Kyler Murray (1.1),QB,1.0
1893,2,12,Isiah Pacheco,Alec Sa,2024,Isiah Pacheco (2.12),RB,1.0
1894,3,1,Kenneth Walker III,Alec Sa,2024,Kenneth Walker III (3.1),RB,1.0
1917,4,12,A.J. Brown,Alec Sa,2024,A.J. Brown (4.12),WR,1.0
1918,5,1,Cooper Kupp,Alec Sa,2024,Cooper Kupp (5.1),WR,1.0
1941,6,12,Tony Pollard,Alec Sa,2024,Tony Pollard (6.12),RB,1.0
1942,7,1,Dalton Kincaid,Alec Sa,2024,Dalton Kincaid (7.1),TE,1.0
1965,8,12,Tyler Bass,Alec Sa,2024,Tyler Bass (8.12),K,1.0
1966,9,1,Bears D/ST,Alec Sa,2024,Bears D/ST (9.1),D/ST,1.0
